# 01. Let's import necessary libraries

In [1]:
import os
import shutil
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt;
import psutil

import sys
import pyemu
import flopy
import swatp_pst
assert "dependencies" in flopy.__file__
assert "dependencies" in pyemu.__file__
assert "dependencies" in swatp_pst.__file__

In [2]:
swatp_pst.__version__

'0.1.1.dev0'

# 00. provide infos

In [3]:
# working directory for mac
# prj_dir = "/Users/seonggyu.park/Documents/projects/jj_test"
# swatp_wd = "/Users/seonggyu.park/Documents/projects/tools/swatp_pst_wf/models/TxtInOut_Imsil_rye_rot_r1"

In [5]:
# working directory for win
prj_dir = "D:\\jj\\opt_2nd"
swatp_wd = "D:\\spark\\jj\\TxtInOut_Imsil_rye_rot_r2"

In [17]:
# calibration period
cal_start = '1/1/2013'
cal_end = '12/31/2023'
# time step
time_step = 'month'
# locations (what our targets)
chs = [1]
obd_file = "singi_obs_q1_colnam.csv"
obd_colnam = "cha01"

# 01. inital setup

In [6]:
from swatp_pst import handler

In [7]:
# copy all necessary files (exes) to your working direcotry
handler.init_setup(prj_dir, swatp_wd)

failed to create 'main_opt' folder, folder already exists ...
path to main_opt folder: D:\spark\jj\opt_2nd\main_opt


In [8]:
# create swatp_con file
handler.create_swatp_pst_con(
    prj_dir, swatp_wd, cal_start, cal_end, chs, time_step=time_step)

,names,vals
0,prj_dir,D:\spark\jj\opt_2nd
1,swatp_wd,D:\spark\jj\TxtInOut_Imsil_rye_rot_r2
2,cal_start,1/1/2013
3,cal_end,12/31/2023
4,chs,[1]
5,time_step,month


In [6]:
os.getcwd()

'D:\\Projects\\Tools\\swatp_pst_wf\\notebooks\\02_workflows'

In [10]:
wd = os.getcwd()

### Let's do initial run!

In [11]:
# you need to run the model first if outputs do not exit.
# pyemu.os_utils.run(r"swatplus.exe", cwd=".")

# 02. Build template files

In [7]:
from swatp_pst.pst_utils import PstUtil

In [8]:
m1 = PstUtil(os.path.join(prj_dir, "main_opt"))

In [9]:
m1.cal_to_tpl_file()

,0,1,2,3,4,5,6,7,8,9,10,11
0,,,,,,,,,,,,
cn2,cn2,pctchg,~ cn2 ~,0,0,0,0,0,0,0,0,0
petco,petco,pctchg,~ petco ~,0,0,0,0,0,0,0,0,0
latq_co,latq_co,pctchg,~ latq_co ~,0,0,0,0,0,0,0,0,0
alpha,alpha,pctchg,~ alpha ~,0,0,0,0,0,0,0,0,0
epco,epco,pctchg,~ epco ~,0,0,0,0,0,0,0,0,0
awc,awc,pctchg,~ awc ~,0,0,0,0,0,0,0,0,0
esco,esco,pctchg,~ esco ~,0,0,0,0,0,0,0,0,0
perco,perco,pctchg,~ perco ~,0,0,0,0,0,0,0,0,0
cn3_swf,cn3_swf,pctchg,~ cn3_swf ~,0,0,0,0,0,0,0,0,0


# 03. Build instruction files

In [15]:
# extract output first
# m1 = SWATpOut(wd)
m1.extract_mon_stf(chs, cal_start, cal_end)

stf_001.txt file has been created...
Finished ...


In [18]:
# m1.stf_obd_to_ins('stf_058.txt', 'rch058',cal_start, cal_end)
m1.stf_obd_to_ins("stf_001.txt", obd_file, obd_colnam, cal_start, cal_end)

stf_001.txt.ins file has been created...


date
2013-01-31    l1 w !cha01_20130131!
2013-02-28    l1 w !cha01_20130228!
2013-03-31    l1 w !cha01_20130331!
2013-04-30    l1 w !cha01_20130430!
2013-05-31    l1 w !cha01_20130531!
                      ...          
2023-08-31    l1 w !cha01_20230831!
2023-09-30    l1 w !cha01_20230930!
2023-10-31    l1 w !cha01_20231031!
2023-11-30    l1 w !cha01_20231130!
2023-12-31                       l1
Name: cha01_ins, Length: 132, dtype: object

# 04. Create PEST control file

In [10]:
io_files = pyemu.helpers.parse_dir_for_io_files('.')
pst = pyemu.Pst.from_io_files(*io_files)

In [11]:
par = pst.parameter_data
par

,parnme,partrans,parchglim,parval1,parlbnd,parubnd,pargp,scale,offset,dercom
alpha,alpha,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1
awc,awc,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1
canmx,canmx,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1
cn2,cn2,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1
cn3_swf,cn3_swf,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1
epco,epco,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1
esco,esco,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1
lat_len,lat_len,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1
latq_co,latq_co,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1
perco,perco,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1


In [12]:
obd = pst.observation_data
obd

,obsnme,obsval,weight,obgnme
cha01_20130131,cha01_20130131,0.3507,1.0,obgnme
cha01_20130228,cha01_20130228,1.0420,1.0,obgnme
cha01_20130331,cha01_20130331,1.7200,1.0,obgnme
cha01_20130430,cha01_20130430,0.4427,1.0,obgnme
cha01_20130531,cha01_20130531,1.1730,1.0,obgnme
...,...,...,...,...
cha01_20230731,cha01_20230731,38.7000,1.0,obgnme
cha01_20230831,cha01_20230831,7.0330,1.0,obgnme
cha01_20230930,cha01_20230930,2.9410,1.0,obgnme
cha01_20231031,cha01_20231031,0.1964,1.0,obgnme


## 05.02 Adjust initial parameter values and their ranges

In [13]:
par = m1.update_par(par)

AttributeError: 'PstUtil' object has no attribute 'update_par'

## 05.03 Assign parameter group name

In [17]:
# set observation group
obd = pst.observation_data
obd

,obsnme,obsval,weight,obgnme
cha01_20130131,cha01_20130131,1.000000e+10,1.0,obgnme
cha01_20130228,cha01_20130228,1.000000e+10,1.0,obgnme
cha01_20130331,cha01_20130331,1.000000e+10,1.0,obgnme
cha01_20130430,cha01_20130430,1.000000e+10,1.0,obgnme
cha01_20130531,cha01_20130531,1.000000e+10,1.0,obgnme
...,...,...,...,...
cha01_20230731,cha01_20230731,1.000000e+10,1.0,obgnme
cha01_20230831,cha01_20230831,1.000000e+10,1.0,obgnme
cha01_20230930,cha01_20230930,1.000000e+10,1.0,obgnme
cha01_20231031,cha01_20231031,1.000000e+10,1.0,obgnme


In [18]:
# Change obd group name
for i in range(len(obd)):
    obd.iloc[i, 3] = obd.iloc[i, 0][:-9]
obd

,obsnme,obsval,weight,obgnme
cha01_20130131,cha01_20130131,1.000000e+10,1.0,cha01
cha01_20130228,cha01_20130228,1.000000e+10,1.0,cha01
cha01_20130331,cha01_20130331,1.000000e+10,1.0,cha01
cha01_20130430,cha01_20130430,1.000000e+10,1.0,cha01
cha01_20130531,cha01_20130531,1.000000e+10,1.0,cha01
...,...,...,...,...
cha01_20230731,cha01_20230731,1.000000e+10,1.0,cha01
cha01_20230831,cha01_20230831,1.000000e+10,1.0,cha01
cha01_20230930,cha01_20230930,1.000000e+10,1.0,cha01
cha01_20231031,cha01_20231031,1.000000e+10,1.0,cha01


## 05.04 Provide actual observed values to control file

In [19]:
os.getcwd()

'D:\\jj\\main_opt'

In [20]:
# Streamflow
stf_obd = pd.read_csv(obd_file,
                       index_col = 0,
                       parse_dates = True,
                       na_values=[-999, '']
                     )
stf_obd = stf_obd[cal_start: cal_end]
stf_obd

,cha01
date,
2013-01-01,1.5
2013-02-01,2.3
2013-03-01,2.7
2013-04-01,1.6
2013-05-01,2.2
...,...
2023-08-01,7.9
2023-09-01,5.3
2023-10-01,1.6


In [ ]:
# Get sub list based on obd order
obd_order = []
for i in obd.obgnme.tolist():
    if i not in obd_order:
        obd_order.append(i)
obd_order

In [ ]:
# get total list from each sub obd, delete na vals
tot_obd = []
for i in obd_order[:1]:
    tot_obd += dtw_obd[i].dropna().tolist()
    print(i)
for i in obd_order[1:]:
    tot_obd += stf_obd[i].dropna().tolist()
    print(i)
len(tot_obd)

In [ ]:
obd.loc[:, 'obsval'] = tot_obd
obd

# 06. Create the control file with settings

We can inspect all control data values using the `pst.control_data.formatted_values` attribute. Values are assigned defaults if not specified. Nice.:

In [21]:
pst.control_data.formatted_values

name
rstfle                        restart
pestmode                   estimation
npar                                0
nobs                                0
npargp                              0
nprior                              0
nobsgp                              0
maxcompdim                          0
ntplfle                             0
ninsfle                             0
precis                         single
dpoint                          point
numcom                              1
jacfile                             0
messfile                            0
obsreref                   noobsreref
rlambda1                 2.000000E+01
rlamfac                 -3.000000E+00
phiratsuf                3.000000E-01
phiredlam                1.000000E-02
numlam                             -7
jacupdate                         999
lamforgive                 lamforgive
derforgive               noderforgive
relparmax                1.000000E+01
facparmax                1.000000E+01
facorig

In [14]:
pst.control_data.noptmax = 0 # replace 0 with "zero" and see what happens
pst.model_command = 'python forward_run.py'

- ### add new PEST++ variables like so:

In [23]:
# check the dictionary again
pst.pestpp_options

{}

In [15]:
pst.write('swatp_dummy.pst', version=2)

noptmax:0, npar_adj:13, nnz_obs:124


- ### You can also read and load the existing pest control file.

In [ ]:
# pst_read = pyemu.Pst(os.path.join(swatmf_model,"mb_zon.pst"))

In [ ]:
pst_read.parameter_data